In [3]:
# libraries
import pandas as pd
import numpy as np
from math import sqrt
import os

In [33]:
# parametros de entrada

# examplo: folder de 2000
folder = r'D:\mestrado\inmet\2000\2000'
ouput_folder = r'D:\mestrado\etr_inmet\etr_inmet'
ano = 2000

# lista de arquivos
list_files = os.listdir(folder)
list_files

# tabela dados estacoes
df_latlon = pd.read_csv(r'D:\mestrado\etr_inmet\etr_inmet\table',
                        delimiter=';',decimal=',',engine='python')



In [32]:
def etr_calc(date,tair_mean,tmin,tmax,ea,esat,rs24h,ws,lat,elev):
    
    # day of the year
    doy = date.timetuple().tm_yday
    d1 = 2*np.pi/365
    d2 = d1*doy
    d3 = np.cos(d2)
    dr = 1 + 0.033*d3

    e1 = 2*np.pi*doy
    e2 = e1/365
    e3 = e2 - 1.39
    e4 = np.sin(e3)
    solar_dec = 0.409*e4
    # lat to radians
    lat_rad = lat*(np.pi)/180

    sun_hour = np.arccos(-np.tan(lat_rad)*np.tan(solar_dec))

    coszen = sun_hour*np.sin(solar_dec)*np.sin(lat_rad)+np.cos(solar_dec)*np.cos(lat_rad)*np.sin(sun_hour)

    gsc = 4.92 # MJ M2 H1
    
    # extraterrestrial radiation
    ra24h = (24/np.pi)*gsc * dr * ( (sun_hour * np.sin(lat_rad)* np.sin(solar_dec)) + 
                                (np.cos(lat_rad) * np.cos(solar_dec) * np.sin(sun_hour)))*11.5740
    
    #pressure
    press =  101.3*(tair_mean/(tair_mean+0.0065*elev))**5.26

    w = (0.14 * ea * press) + 2.1

    W =1
    tao  = 0.35 + 0.627 * np.exp(((-0.00146 * press)/(W * coszen)) - (0.075 * (W / coszen)**0.4))

    rso24h = ra24h * tao

    rnl24h = 4.901E-9*((tmax**4+tmin**4)/2)*(0.34-0.14*np.sqrt(ea))*(1.35*(rs24h/rso24h)-0.35)*(11.5740)

    albedo = 0.23
    rn24h = ((1 - albedo) * rs24h) - rnl24h

    tair = tair_mean -273.15
    delta = (2503*(np.exp((17.27*tair)/(tair+237.3))))/(tair +237.3)**2

    le_vap = 2.501-0.002361*tair

    psi = 0.0016286*press/le_vap
    
    z_alt=10
    ws = ws*((4.87)/(np.log(67.8*z_alt-5.42)))

    Cn = 1600
    Cd = 0.38
    etr = (0.408*(delta)*(rn24h/11.6)+((psi)*((Cn)/(tair+273)))*((ws)*(esat-ea)))/(delta+psi*(1+Cd*ws))
    
    return etr

In [35]:

# loop para calcular a etr
for file in list_files:
    
    df = pd.read_csv(folder+'\\'+file,delimiter=';',skiprows=8,decimal=',', engine='python')
    df = df.replace(-9999,np.nan)
    #nome estacao
    estacao = file.split('_')[3]
    #tabela dados da estacao
    df_estacao = df_latlon[df_latlon['CD_ESTACAO'] ==estacao]

    # padronizando data
    if 'DATA (YYYY-MM-DD)' in df.keys().tolist():
        df['Data'] = df['DATA (YYYY-MM-DD)']
        df = df.drop(columns=['DATA (YYYY-MM-DD)'])
                              
    # selecionando as variaveis
    df2= pd.DataFrame()
    df2['date'] = df['Data']
    df2['precipt'] = df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)']
    try:
        df2['rad_global'] = df['RADIACAO GLOBAL (Kj/m²)'] 
    except:
        df2['rad_global'] = df['RADIACAO GLOBAL (KJ/m²)']
        
    df2['temp_mean'] = df['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)']
    df2['temp_min'] = df['TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)']
    df2['temp_max'] = df['TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)']
    df2['rh'] = df['UMIDADE RELATIVA DO AR, HORARIA (%)']
    df2['wind'] = df['VENTO, VELOCIDADE HORARIA (m/s)']
    df2['datetime'] = pd.to_datetime(df2['date'])
    
    # calculando medias, minimas e maximas                          
    df2_mean = df2.set_index('datetime').resample('1D').mean().reset_index()
    df2_max = df2.set_index('datetime').resample('1D').max().reset_index()
    df2_sum = df2.set_index('datetime').resample('1D').sum().reset_index()
    df2_min =  df2.set_index('datetime').resample('1D').min().reset_index()
    
    # calculando variaveis ncessarias para a etr
    df_station = pd.DataFrame()
    df_station['datetime'] = df2_mean['datetime']
    df_station['tmean_inmet'] = df2_mean['temp_mean']+273.15
    df_station['tmin_inmet'] = df2_min['temp_min']+273.15
    df_station['tmax_inmet'] = df2_max['temp_max']+273.15
    df_station['ea_inmet'] = 0.6108*(np.exp((17.27*(df2_min['temp_min']))/(df2_min['temp_min']+237.3)))
    df_station['wind_inmet'] = df2_mean['wind']
    
    # calculando esat
    tmin = df2_min['temp_min']
    tmax = df2_max['temp_max']
    
    df_station['esat_inmet'] = (0.6108*(np.exp((17.27*tmin)/(tmin+237.3)))+0.6108*(np.exp((17.27*tmax)/(tmax+237.3))))/2
    df_station['global_rad_inmet'] = df2_sum['rad_global']*1000/86400
    
    k = 0
    list_inmet_etr = []
    for date in df_station['datetime']:
        #global variables
        lat = df_estacao['VL_LATITUDE'].values[0]
        elev = df_estacao['VL_ALTITUDE'].values[0]
    
        # inmet variables
        tair_mean  = df_station['tmean_inmet'][k]
        tmin = df_station['tmin_inmet'][k]
        tmax = df_station['tmax_inmet'][k]
        ea =  df_station['ea_inmet'][k]
        esat = df_station['esat_inmet'][k]
        rs24h = df_station['global_rad_inmet'][k]
        ws = df_station['wind_inmet'][k]
        
        k=k+1
    
        etr_inmet = etr_calc(date,tair_mean,tmin,tmax,ea,esat,rs24h,ws,lat,elev)
        list_inmet_etr.append(etr_inmet)
        
    df_etr = pd.DataFrame()
    df_etr['date'] = df_station['datetime']
    df_etr['etr_inmet'] = list_inmet_etr
    
    df_etr.to_excel(ouput_folder+'\\'+str(estacao)+'_'+str(ano)+'.xlsx')
    